# 异步编程

python是默认的全同步实现,它的异步都是通过框架实现的,比如gevent,tornad,在3.4之后python加入了async标准包实现了语言级别的异步,但貌似买帐的人不多,也算是历史遗留问题了.

所谓"异步",简单说就是一个任务分成两段,先执行第一段,然后转而执行其他任务,等做好了准备再回过头执行第二段.

比如有一个任务是读取文件进行处理,任务的第一段是向操作系统发出请求,要求读取文件.然后程序执行其他任务,等到操作系统返回文件再接着执行任务的第二段(处理文件).这种不连续的执行就叫做异步.

相应地连续的执行就叫做同步.由于是连续执行不能插入其他任务,所以操作系统从硬盘读取文件的这段时间程序只能干等着.

Js没有这些包袱,所以在异步方案方便确实做得比python好,js中异步处理并发主要手段是:

+ 回调函数与Event对象

+ Promise对象

+ 协程


## 回调函数与Event对象

这是最原始也是最好理解的手段.所谓回调函数就是把任务的第二段单独写在一个函数里面,等到重新执行这个任务的时候就直接调用这个函数.它的英语名字`callback`,直译过来就是"重新调用".js早期到处都是回调函数.python中的`tornado`框架就是异步回调实现的服务器

在js中这是非常常见的技术,比如读取一个文件:

In [1]:
import fs from "fs"

In [2]:
fs.readFile('./README.md', 'utf-8',function (err, data) {
  if (err) throw err
  console.log(data.slice(0,20))
})

# Javascript基础语法

ES


上面代码中,readFile函数的第三个参数就是回调函数,也就是任务的第二段.等到操作系统返回了`./README.md`这个文件以后回调函数才会执行.

一个有趣的问题是为什么Node.js约定回调函数的第一个参数必须是错误对象err(如果没有错误，该参数就是null)？原因是执行分成两段,在这两段之间抛出的错误程序无法捕捉,只能当作参数传入第二段.

### Event对象

Event对象代表事件的状态,比如事件在其中发生的元素,键盘按键的状态,鼠标的位置,鼠标按钮的状态等.

事件通常与回调函数结合使用,这与tkinter中类似.

我们需要将回调函数绑定到Event对应的方法上,一旦事件被执行,在执行完事件后会构造一个Event对象并将它传入绑定的回调函数中执行.


### 回调的缺点

回调函数本身并没有问题,它的问题出现在多个回调函数嵌套.假定读取A文件之后再读取B文件,代码如下:

```js
fs.readFile(fileA, function (err, data) {
  fs.readFile(fileB, function (err, data) {
    // ...
  })
})
```

如果依次读取多个文件就会出现多重嵌套.代码不是纵向发展而是横向发展,很快就会乱成一团从而无法管理.这种情况就称为"回调函数噩梦"(callback hell)

## Promise

promise翻译成中文就是保证,是异步编程的一种解决方案,比传统的解决方案——回调函数和事件——更合理和更强大.它可以通过链式写法将回调函数穿起来,这样就不再是一层套一层的嵌套写法,回调函数可以像平铺一样的顺序展开写下,这样就更加便于维护.


Promise其实可以理解为规划,如果说回调是面向对象的,那Promise就是面向全局的,它把每种可能都描述在一起,而不是分开来给不同的地方.像上面读取文件的例子.我们可以使用`fs`的Promise接口.

In [4]:
fs.promises.readFile('./README.md', 'utf-8')
    .then(data=>console.log(data.slice(0,20)))
    .catch(error=>console.error(error))

# Javascript基础语法

ES


## 协程

传统的编程语言早有异步编程的解决方案(其实是多任务的解决方案).其中有一种叫做"协程"(coroutine),意思是多个程序互相协作完成异步任务.

协程有点像函数,又有点像线程.它的运行流程大致如下:

1. 协程A开始执行

2. 协程A执行到一半进入暂停,执行权转移到协程B.

3. 一段时间后协程B交还执行权.

4. 协程A恢复执行

上面流程的协程A就是异步任务,因为它分成两段(或多段)执行.

由此可见我们需要这么几个部件就可以实现协程

+ 一个可以转入转出执行权,且可以记录执行状态的结构
+ 一个分配什么时间谁来执行的结构


### 用生成器构造协程

可以转入转出执行权的结构我们很容易就可以想到利用生成器.生成器可以利用`yield`和`next`转入转出执行权,还可以传入传出数据.

像python中协程就是从生成器中衍生出来的,js也不例外.

举例来说,读取文件的协程写法如下:

```js
function *asnycJob() {
  // ...其他代码
  let f = yield readFile(fileA);
  // ...其他代码
}
```

上面代码的函数asyncJob是一个协程,它的奥妙就在其中的`yield`命令,它表示执行到此处执行权将交给其他协程,也就是说`yield`命令是异步两个阶段的分界线.

协程遇到`yield`命令就暂停,等到执行权返回,再从暂停的地方继续往后执行.它的最大优点就是代码的写法非常像同步操作,如果去除`yield`命令简直一模一样.

生成器函数是协程在ES6的实现,最大特点就是可以交出函数的执行权(即暂停执行).

整个生成器就是一个封装的异步任务,或者说是异步任务的容器.异步操作需要暂停的地方都用`yield`语句注明.生成器的执行方法如下:

```js

function* gen(x){
  let y = yield x + 2;
  return y;
}

var g = gen(1);
g.next() // { value: 3, done: false }
g.next() // { value: undefined, done: true }
```

上面代码中调用生成器会返回一个内部指针(即遍历器)g.这是生成器不同于普通函数的另一个地方,即执行它不会返回结果,而返回的是指针对象.调用指针g的`next`方法会移动内部指针(即执行异步任务的第一段),指向第一个遇到的yield语句.上例是执行到`x + 2`为止.

换言之`next`方法的作用是分阶段执行生成器.每次调用`next`方法会返回一个对象,表示当前阶段的信息(value属性和done属性).value属性是`yield`语句后面表达式的值,表示当前阶段的值;done属性是一个布尔值,表示生成器是否执行完毕,即是否还有下一个阶段.


#### 生成器函数的数据交换和错误处理

生成器函数可以暂停执行和恢复执行,这是它能封装异步任务的根本原因.除此之外,它还有两个特性,使它可以作为异步编程的完整解决方案:函数体内外的数据交换和错误处理机制

next方法返回值的value属性,是生成器函数向外输出数据;next方法还可以接受参数,这是向生成器函数体内输入数据

In [17]:
function* gen(x){
  let y = yield x + 2;
  return y;
}
let g = gen(1);
console.log(g.next()) // { value: 3, done: false }
console.log(g.next(2)) // { value: 2, done: true }

{ value: 3, done: false }
{ value: 2, done: true }


上面代码中,第一个next方法的value属性,返回表达式x + 2的值.第二个next方法带有参数2,这个参数可以传入生成器函数,作为上个阶段异步任务的返回结果,被函数体内的变量y接收.因此这一步的 value 属性,返回的就是2(变量y的值)

生成器函数内部还可以部署错误处理代码,捕获函数体外抛出的错误

In [1]:
function* gen(x){
  try {
    let y = yield x + 2;
  } catch (e){
    console.log(e);
  }
  return y;
}

let g = gen(1);
g.next();
g.throw('出错了');
// 出错了


出错了


ReferenceError: y is not defined

上面代码的最后一行,生成器函数体外,使用指针对象的throw方法抛出的错误,可以被函数体内的try ...catch代码块捕获.这意味着出错的代码与处理错误的代码,实现了时间和空间上的分离,这对于异步编程无疑是很重要的

### 异步任务的封装

下面看看如何使用生成器函数以执行一个真实的异步任务。

In [10]:
import fetch from 'node-fetch'
function* gen(){
  let url = 'https://api.github.com/users/github'
  let result = yield fetch(url)
  console.log(result.bio)
}

执行

In [11]:
let g = gen()
let result = g.next()

result.value.then(function(data){
  return data.json()
}).then(function(data){
  g.next(data)
})

How people build software.


上面代码中生成器函数封装了一个异步操作,该操作先读取一个远程接口,然后从JSON格式的数据解析信.就像前面说过的这段代码非常像同步操作,除了加上了yield命令.

上面代码中首先执行生成器函数,获取遍历器对象,然后使用`next`方法(第二行),执行异步任务的第一阶段.由于Fetch模块返回的是一个Promise对象,因此要用then方法调用下一个`next`方法.

可以看到虽然生成器函数将异步操作表示得很简洁,但是流程管理却不方便(即何时执行第一阶段,何时执行第二阶段).

### Thunk函数

Thunk函数早在上个世纪60年代就诞生了.那时编程语言刚刚起步,计算机学家还在研究编译器怎么写比较好.一个争论的焦点是"求值策略"--函数的参数到底应该何时求值

+ 一种意见是"传值调用"(call by value),即在进入函数体之前就计算表达式的值,再将这个值传入函数f.C语言就采用这种策略.

    ```
    f(x + 5)
    // 传值调用时，等同于
    f(6)
    ```
    
+ 另一种意见是"传名调用"(call by name),即直接将表达式x + 5传入函数体,只在用到它的时候求值.Haskell语言采用这种策略.

    ```javascript
    function f(x){return x*2}
    f(x + 5)
    // 传名调用时，等同于
    (x + 5) * 2
    ```

传值调用和传名调用哪一种比较好？回答是各有利弊.传值调用比较简单,但是对参数求值的时候实际上还没用到这个参数,有可能造成性能损失.

```javascript
function f(a, b){
  return b;
}

f(3 * x * x - 2 * x - 1, x)
```

上面代码中函数f的第一个参数是一个复杂的表达式,但是函数体内根本没用到.对这个参数求值实际上是不必要的.因此有一些计算机学家倾向于"传名调用"即只在执行时求值。

编译器的"传名调用"实现,往往是将参数放到一个临时函数之中,再将这个临时函数传入函数体.这个临时函数就叫做Thunk函数.

```js
function f(m){
  return m * 2;
}

f(x + 5);

// 等同于

let thunk = function () {
  return x + 5;
};

function f(thunk){
  return thunk() * 2;
}
```

上面代码中函数f的参数x + 5被一个函数替换了.凡是用到原参数的地方对Thunk函数求值即可. 这就是Thunk函数的定义.它是"传名调用"的一种实现策略,用来替换某个表达式.

### JavaScript语言的Thunk函数

JavaScript语言是传值调用,它的Thunk函数含义有所不同.在JavaScript语言中,Thunk函数替换的不是表达式,而是多参数函数,将其替换成单参数的版本.且只接受回调函数作为参数。

```js
// 正常版本的readFile（多参数版本）
fs.readFile(fileName, callback)

// Thunk版本的readFile（单参数版本）
let readFileThunk = Thunk(fileName)
readFileThunk(callback)

let Thunk = function (fileName){
  return function (callback){
    return fs.readFile(fileName, callback)
  }
}
```

上面代码中fs模块的readFile方法是一个多参数函数,两个参数分别为文件名和回调函数.经过转换器处理它变成了一个单参数函数,只接受回调函数作为参数.这个单参数版本就叫做Thunk函数.

任何函数只要参数有回调函数就能写成Thunk函数的形式.下面是一个简单的Thunk函数转换器.

```js
let Thunk = function(fn){
  return function (){
    let args = Array.prototype.slice.call(arguments)
    return function (callback){
      args.push(callback)
      return fn.apply(this, args)
    }
  }
}
```

使用上面的转换器，生成fs.readFile的Thunk函数。

```js
let readFileThunk = Thunk(fs.readFile)
readFileThunk(fileA)(callback)
```

### Thunkify模块

生产环境的转换器建议使用Thunkify模块.

```bash
npm install thunkify
```
    
使用方式如下:



```javascript
let thunkify = require('thunkify');
let fs = require('fs');

let read = thunkify(fs.readFile);
read('package.json')(function(err, str){
  // ...
})

```
Thunkify的源码与上一节那个简单的转换器非常像.

```js
function thunkify(fn){
  return function(){
    let args = new Array(arguments.length)
    let ctx = this

    for(let i = 0; i < args.length; ++i) {
      args[i] = arguments[i]
    }

    return function(done){
      let called

      args.push(function(){
        if (called) return
        called = true
        done.apply(null, arguments)
      })

      try {
        fn.apply(ctx, args)
      } catch (err) {
        done(err)
      }
    }
  }
}

```

它的源码主要多了一个检查机制,变量called确保回调函数只运行一次.这样的设计与下文的生成器函数相关.请看下面的例子:

```js
function f(a, b, callback){
  let sum = a + b;
  callback(sum);
  callback(sum);
}

let ft = thunkify(f);
let print = console.log.bind(console);
ft(1, 2)(print);
// 3
```
上面代码中,由于thunkify只允许回调函数执行一次,所以只输出一行结果.

### 生成器函数的流程管理

Thunk函数有什么用?回答是以前确实没什么用,但是ES6有了生成器函数,Thunk函数现在可以用于生成器函数的自动流程管理.

以读取文件为例,下面的生成器函数封装了两个异步操作:

In [1]:
import fs from "fs"
import thunkify from'thunkify'
    
let readFile = thunkify(fs.readFile)

let gen = function* (){
  let r1 = yield readFile('./README.md',"utf-8")
  console.log(r1.toString())
  let r2 = yield readFile('./code/circle.js',"utf-8")
  console.log(r2.toString())
}

上面代码中yield命令用于将程序的执行权移出生成器函数,那么就需要一种方法将执行权再交还给生成器函数.

这种方法就是Thunk函数.因为它可以在回调函数里将执行权交还给生成器函数.为了便于理解,我们先看如何手动执行上面这个生成器函数.

In [2]:
let g = gen()

let r1 = g.next()
r1.value(function(err, data){
  if (err) throw err
  let r2 = g.next(data)
  r2.value(function(err, data){
    if (err) throw err
    g.next(data)
  })
})

# Javascript基础语法

ES6已经在稍微现代点的浏览器和node.js中实装,本文所讲的语法内容基本以ES6为基准,包括:

+ 基本语法和代码结构
+ 变量,值与运算
+ 从运算符到函数
+ 迭代器和生成器
+ 基本容器
+ 异步编程
+ 面向对象编程
+ 模块化编程
+ 特殊对象和特殊接口


如果想要更加细节的js语法知识,可以看[MDN上的文档](https://developer.mozilla.org/zh-CN/docs/Web/JavaScript/Guide)
area = (r) => {
    return Math.PI * Math.pow(r, 2)
}
circumference = (r) => {
    return 2 * Math.PI * r
}

createpoint = function (x, y) {
    let point = function (x, y) {
        this.x = x
        this.y = y
        this.add = function (that) {
            let x = this.x + that.x
            let y = this.y + that.y
            return new point(x, y)
        }
    }
    //protopye中定义类方法和重载一些方法
    point.prototype.toString = function () {
        return '(' + this.x + ', ' + this.y + ')'
    }
    return new point(x, y)
}

export {
    area,
    circumference,
    createpoint
}


上面代码中变量g是生成器函数的内部指针,表示目前执行到哪一步.next方法负责将指针移动到下一步,并返回该步的信息(value属性和done属性).

仔细查看上面的代码可以发现生成器函数的执行过程其实是将同一个回调函数反复传入next方法的value属性.这使得我们可以用递归来自动完成这个过程.

### Thunk函数的自动流程管理

Thunk函数真正的威力在于可以自动执行生成器函数.下面就是一个基于Thunk函数的生成器执行器.

In [1]:
import * as fs from "fs"
import thunkify from'thunkify'

let readFile = thunkify(fs.readFile)

let gen = function* (){
  let r1 = yield readFile('./README.md',"utf-8")
  console.log(r1.toString())
  let r2 = yield readFile('./code/circle.js',"utf-8")
  console.log(r2.toString())
}


function run(fn) {
  let gen = fn()

  function next(err, data) {
    let result = gen.next(data)
    if (result.done) return
    result.value(next)
  }

  next()
}

run(gen)

# Javascript基础语法

ES6已经在稍微现代点的浏览器和node.js中实装,本文所讲的语法内容基本以ES6为基准,包括:

+ 基本语法和代码结构
+ 变量,值与运算
+ 从运算符到函数
+ 迭代器和生成器
+ 基本容器
+ 异步编程
+ 面向对象编程
+ 模块化编程
+ 特殊对象和特殊接口


如果想要更加细节的js语法知识,可以看[MDN上的文档](https://developer.mozilla.org/zh-CN/docs/Web/JavaScript/Guide)
area = (r) => {
    return Math.PI * Math.pow(r, 2)
}
circumference = (r) => {
    return 2 * Math.PI * r
}

createpoint = function (x, y) {
    let point = function (x, y) {
        this.x = x
        this.y = y
        this.add = function (that) {
            let x = this.x + that.x
            let y = this.y + that.y
            return new point(x, y)
        }
    }
    //protopye中定义类方法和重载一些方法
    point.prototype.toString = function () {
        return '(' + this.x + ', ' + this.y + ')'
    }
    return new point(x, y)
}

export {
    area,
    circumference,
    createpoint
}


上面代码的run函数就是一个生成器函数的自动执行器.内部的next函数就是Thunk的回调函数.next函数先将指针移到生成器函数的下一步(gen.next方法),然后判断生成器函数是否结束(result.done 属性).如果没结束就将next函数再传入Thunk函数(result.value属性);否则就直接退出.

有了这个执行器,执行生成器方便多了,不管有多少个异步操作直接传入run函数即可.当然前提是每一个异步操作都要是Thunk函数,也就是说跟在yield命令后面的必须是Thunk函数.

In [2]:
let gen1 = function* (){
  let f1 = yield readFile('./README.md')
  let f2 = yield readFile('./code/circle.js')
}

run(gen1)

上面代码中函数gen1封装了n个异步的读取文件操作,只要执行run函数这些操作就会自动完成.这样一来异步操作不仅可以写得像同步操作,而且一行代码就可以执行.

Thunk函数并不是生成器函数自动执行的唯一方案.因为自动执行的关键是必须有一种机制自动控制生成器函数的流程,接收和交还程序的执行权.回调函数可以做到这一点,Promise对象也可以做到这一点.

### co模块


co模块是著名程序员TJ Holowaychuk于2013年6月发布的一个小工具,用于生成器函数的自动执行.

比如有一个生成器函数用于依次读取两个文件

```js
let gen = function* (){
  let f1 = yield readFile('/etc/fstab')
  let f2 = yield readFile('/etc/shells')
  console.log(f1.toString())
  console.log(f2.toString())
};
```
co模块可以让你不用编写生成器函数的执行器.

```js
let co = require('co')
co(gen)
```

上面代码中生成器函数只要传入co函数就会自动执行.

co函数返回一个Promise对象,因此可以用then方法添加回调函数.

```js
co(gen).then(function (){
  console.log('Generator 函数执行完成');
})
```

上面代码中等到生成器函数执行结束就会输出一行提示.

为什么co可以自动执行生成器函数？

前面说过生成器就是一个异步操作的容器,它的自动执行需要一种机制,当异步操作有了结果时能够自动交回执行权.

两种方法可以做到这一点:

+ `回调函数`将异步操作包装成Thunk函数,在回调函数里面交回执行权.

+ `Promise`将异步操作包装成Promise对象,用then方法交回执行权.


co模块其实就是将两种自动执行器(Thunk函数和Promise对象)包装成一个模块.使用co的前提条件是生成器函数的yield命令后面只能是Thunk函数或Promise对象.

上面已经介绍了基于Thunk函数的自动执行器.下面来看基于Promise对象的自动执行器.这是理解co模块必须的.

还是沿用上面的例子.首先把fs模块的readFile方法包装成一个Promise对象.

In [1]:
let readFile1 = function (fileName){
  return new Promise(function (resolve, reject){
    fs.readFile(fileName, function(error, data){
      if (error) reject(error)
      resolve(data)
    })
  })
}

let gen2 = function* (){
  let f1 = yield readFile1('./README.md')
  let f2 = yield readFile1('./code/circle.js')
  console.log(f1.toString())
  console.log(f2.toString())
}

然后，手动执行上面的生成器函数。

In [2]:
let g = gen2();

g.next().value.then(function(data){
  g.next(data).value.then(function(data){
    g.next(data)
  })
})

# Javascript基础语法

ES6已经在稍微现代点的浏览器和node.js中实装,本文所讲的语法内容基本以ES6为基准,包括:

+ 基本语法和代码结构
+ 变量,值与运算
+ 从运算符到函数
+ 迭代器和生成器
+ 基本容器
+ 异步编程
+ 面向对象编程
+ 模块化编程
+ 特殊对象和特殊接口


如果想要更加细节的js语法知识,可以看[MDN上的文档](https://developer.mozilla.org/zh-CN/docs/Web/JavaScript/Guide)
area = (r) => {
    return Math.PI * Math.pow(r, 2)
}
circumference = (r) => {
    return 2 * Math.PI * r
}

createpoint = function (x, y) {
    let point = function (x, y) {
        this.x = x
        this.y = y
        this.add = function (that) {
            let x = this.x + that.x
            let y = this.y + that.y
            return new point(x, y)
        }
    }
    //protopye中定义类方法和重载一些方法
    point.prototype.toString = function () {
        return '(' + this.x + ', ' + this.y + ')'
    }
    return new point(x, y)
}

export {
    area,
    circumference,
    createpoint
}


手动执行其实就是用then方法,层层添加回调函数.

In [3]:
function co(gen) {
  let ctx = this
  return new Promise(function(resolve, reject) {
  })
}

在返回的Promise对象里面,co先检查参数gen是否为生成器函数.如果是就执行该函数,得到一个内部指针对象;如果不是就返回,并将Promise对象的状态改为resolved.

In [4]:
function co(gen) {
  let ctx = this

  return new Promise(function(resolve, reject) {
    if (typeof gen === 'function') gen = gen.call(ctx)
    if (!gen || typeof gen.next !== 'function') return resolve(gen)
  })
}

接着co将生成器函数的内部指针对象的next方法包装成onFulfilled函数.这主要是为了能够捕捉抛出的错误.

In [5]:
function co(gen) {
  let ctx = this

  return new Promise(function(resolve, reject) {
    if (typeof gen === 'function') gen = gen.call(ctx)
    if (!gen || typeof gen.next !== 'function') return resolve(gen)

    onFulfilled()
    function onFulfilled(res) {
      let ret
      try {
        ret = gen.next(res)
      } catch (e) {
        return reject(e)
      }
      next(ret)
    }
  })
}

最后就是关键的next函数,它会反复调用自身.

In [6]:
function next(ret) {
  if (ret.done) return resolve(ret.value)
  let value = toPromise.call(ctx, ret.value)
  if (value && isPromise(value)) return value.then(onFulfilled, onRejected)
  return onRejected(new TypeError('You may only yield a function, promise, generator, array, or object, '
    + 'but the following object was passed: "' + String(ret.value) + '"'))
}

上面代码中next函数的内部代码一共只有四行命令.

+ 第一行检查当前是否为生成器函数的最后一步,如果是就返回.

+ 第二行确保每一步的返回值是 Promise 对象.

+ 第三行使用then方法,为返回值加上回调函数,然后通过`onFulfilled`函数再次调用next函数.

+ 第四行,在参数不符合要求的情况下(参数非Thunk函数和Promise对象),将Promise对象的状态改为rejected从而终止执行.


### 处理并发的异步操作

co支持并发的异步操作,即允许某些操作同时进行,等到它们全部完成才进行下一步.

这时要把并发的操作都放在数组或对象里面,跟在yield语句后面.



```js
// 数组的写法
co(function* () {
  let res = yield [
    Promise.resolve(1),
    Promise.resolve(2)
  ];
  console.log(res);
}).catch(onerror);

// 对象的写法
co(function* () {
  let res = yield {
    1: Promise.resolve(1),
    2: Promise.resolve(2),
  };
  console.log(res);
}).catch(onerror);
//下面是另一个例子。

co(function* () {
  let values = [n1, n2, n3];
  yield values.map(somethingAsync);
});

function* somethingAsync(x) {
  // do something async
  return y
}
```

上面的代码允许并发三个somethingAsync异步操作,等到它们全部完成,才会进行下一步.

### 异步协程语法async...await

本质来说`async/await`是一种用于处理JS异步操作的语法糖,利用协程帮助我们利用同步语法摆脱回调函数使用Promise,从而编写更加优雅的代码.

我们的阅读例子改为异步协程语法实现如下:

In [9]:
(async ()=>{
    try{
        let data = await fs.promises.readFile('./README.md', 'utf-8')
        console.log(data.slice(0,20))
    }catch(error){
        console.error(error)
    }
})()

# Javascript基础语法

ES


## 常用的支持promise的第三方异步库

+ http客户端<https://github.com/axios/axios>
+ http服务端<https://github.com/koajs/koa>
+ 文件系统<https://www.npmjs.com/package/fs-extra>
+ orm<https://github.com/sequelize/sequelize>
